# Grupo 1 - Smog predicition

## Inspección y tratamiento de los datos proporcionados

Inicialmente se va a analizar el dataset proporcionado, buscando que variables pueden ser útiles para realizar el reto. Para ello, comenzamos por importar los datos.

In [ ]:
import pandas as pd

df = pd.read_csv('data/train.csv')
df.head()

De un primer vistazo, algunos datos que pueden ser de utilidad para realizar la predicción podrían ser "Engine Size (L)", "Cylinders", "Transmission", "Fuel Type", "Comb (L/100 km)" y "CO2 Emissions (g/km)". 

In [ ]:
df.info()

Hemos decidido dividir la columna "Transmission" para poder obtener el número de marchas ("Gears") y el tipo de transmisión ("Transmission") de cada coche de forma separada para facilitar la conversión de esta segunda propiedad a valores numéricos.

In [ ]:
df['Gears'] = df['Transmission'].str.extract('(\d+)')
df['Gears'] = pd.to_numeric(df['Gears'], errors='coerce')
df['Transmission'] = df['Transmission'].str.extract('(\D+)')
df.head()

Es necesario transformar los valores categóricos a numéricos para poder visualizar los datos y entrenar el modelo correctamente.

In [ ]:
#Fuel Type
df.loc[df["Fuel Type"] == "X", "Fuel Type"] = 0
df.loc[df["Fuel Type"] == "Z", "Fuel Type"] = 1
df.loc[df["Fuel Type"] == "D", "Fuel Type"] = 2
df.loc[df["Fuel Type"] == "E", "Fuel Type"] = 3
df.loc[df["Fuel Type"] == "N", "Fuel Type"] = 4

#Transmission
df.loc[df["Transmission"] == "A", "Transmission"] = 0
df.loc[df["Transmission"] == "AM", "Transmission"] = 1
df.loc[df["Transmission"] == "AS", "Transmission"] = 2
df.loc[df["Transmission"] == "AV", "Transmission"] = 3
df.loc[df["Transmission"] == "M", "Transmission"] = 4


#Vehicle Class
df.loc[df["Vehicle Class"] == "Compact", "Vehicle Class"] = 0
df.loc[df["Vehicle Class"] == "Full-size", "Vehicle Class"] = 1
df.loc[df["Vehicle Class"] == "Mid-size", "Vehicle Class"] = 2
df.loc[df["Vehicle Class"] == "Minicompact", "Vehicle Class"] = 3
df.loc[df["Vehicle Class"] == "Minivan", "Vehicle Class"] = 4
df.loc[df["Vehicle Class"] == "Minicompact", "Vehicle Class"] = 5
df.loc[df["Vehicle Class"] == "Pickup truck: Small", "Vehicle Class"] = 6
df.loc[df["Vehicle Class"] == "Pickup truck: Standard", "Vehicle Class"] = 7
df.loc[df["Vehicle Class"] == "SUV: Small", "Vehicle Class"] = 8
df.loc[df["Vehicle Class"] == "SUV: Standard", "Vehicle Class"] = 9
df.loc[df["Vehicle Class"] == "Special purpose vehicle", "Vehicle Class"] = 10
df.loc[df["Vehicle Class"] == "Station wagon: Mid-size", "Vehicle Class"] = 11
df.loc[df["Vehicle Class"] == "Station wagon: Small", "Vehicle Class"] = 12
df.loc[df["Vehicle Class"] == "Subcompact", "Vehicle Class"] = 13
df.loc[df["Vehicle Class"] == "Two-seater", "Vehicle Class"] = 14

A continuación, comprobaremos si existen valores nulos para completarlos o eliminarlos según proceda.

In [ ]:
df.isnull().sum()

Al buscar si al tratar los datos hay algun valor nulo encontramos que en la columna "Gears" hay 22 valores nulos, por lo que hemos decidido completar los datos sin marchas asociadas con la media de "Gears" en los otros datos.

In [ ]:
df['Gears'] = df['Gears'].fillna(df['Gears'].mean())
df.isnull().sum()

Al analizar el dataset hemos detectado que la columna de "Model Year" es igual en todos los datos, por lo que hemos decidido suprimirla al no aportar información relevante para entrenar el modelo.

In [ ]:
df.drop("Model Year", axis=1, inplace=True)
df.head()

También, hemos decidido eliminar las columnas de "Make" y "Model" porque no resultan útiles, al no aportar información relevante para calcular la clasificación de "Smog". Además, así podemos realizar un modelo menos complejo al tener menos características a ajustar.

In [ ]:
df.drop("Make", axis=1, inplace=True)
df.drop("Model", axis=1, inplace=True)
df.head()

Del mismo modo, se eliminará la columna "Comb (mpg)" al ser una métrica equivalente a la anterior, pero en unidades del sistema anglosajón.

In [ ]:
df.drop("Comb (mpg)", axis=1, inplace=True)
df.head()

Finalmente, eliminamos las columnas "Fuel Consumption City (L/100Km)" y "Hwy (L/100Km)" ya que estos datos se han combinado en la columna "Comb (L/100Km)", que será la que se usará como media del consumo de cada coche en ciudad y autopista en nuestro estudio. 

In [ ]:
df.drop("Fuel Consumption City (L/100 km)", axis=1, inplace=True)
df.drop("Hwy (L/100 km)", axis=1, inplace=True)
df.head()

### Visualización de los datos

En este apartado, generaremos algunas gráficas para ilustrar los datos de los que disponemos y obtener así relaciones interesantes que nos proporcionen pistas para el entrentamiento de nuestro modelo.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)

%matplotlib inline

Comenzaremos con un *scatter plot* de todos los parámetros.

In [ ]:
g = sns.PairGrid(df, hue="Smog")
g.map_diag(plt.hist)
g.map_offdiag(plt.scatter)
g.add_legend()

Tras observar los gráficos de las diferentes características correladas podemos deducir que las principales variables para la predicción son "Gears", "Fuel Type", "Cylinders" y "Transmission". Sin embargo, una de las variables que parecía más prometedoras, "Engine Size (L)", finalmente ha resultado ser de las menos correladas con el resto.

A partir de esta primera visualización de los datos, y conociendo las propiedades más interesantes de nuestro dataframe, podemos obtener diversas gráficas que nos ayuden a comprender la relación entre las diferentes variables y su categoría de emisión de *smog*.

In [ ]:
sns.FacetGrid(df, col='Smog').map(plt.hist, "Cylinders", color="steelblue")

En este segundo gráfico puede observarse el número de cilindros de los vehículos en función de su clasificación de *smog*, lo que nos desvela que, en términos generales, aquellos vehículos con mejor clasificación son los que cuentan con un menor número de estos.

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 7))

plt.subplot(1, 2, 1)
transmission_types = ['A', 'AM', 'AS', 'AV', 'M']
smog_per_trans = df.groupby(['Transmission','Smog']).size().unstack(['Transmission']).plot(
    kind='bar', stacked=True, rot=0, subplots=False, ax=plt.gca())
smog_per_trans.legend(labels=transmission_types)
smog_per_trans.set_xlabel('Smog category')
smog_per_trans.set_title('Smog classification per Transmission Type')


plt.subplot(1, 2, 2)
fuel_types = ['Regular', 'Premium', 'Diesel', 'Ethanol']
smog_per_fuel = df.groupby(['Fuel Type','Smog']).size().unstack(['Fuel Type']).plot(
    kind='bar', stacked=True, rot=0, subplots=False, ax=plt.gca()) 
smog_per_fuel.legend(labels=fuel_types)
smog_per_fuel.set_xlabel('Smog category')
smog_per_fuel.set_title('Smog classification per Fuel Type')

plt.tight_layout()
plt.show()

En estas dos gráficas se ha representado la clasificación de emisión de *smog* en función del tipo de transmisión y del tipo de combustible empleado por el vehículo.

En la primera de ellas podemos observar que los vehículos con mejor clasificación medioambiental son aquellos que cuentan con cajas de cambios de tipo `A` y `AM`, además de un pequeño grupo que cuenta con cajas de tipo `M` y `AS`. Ningún vehículo con esta calificación cuenta con una caja de tipo `AV`. Por otro lado, las cajas de tipo `AV` y `AS`son mayoritarias entre los vehículos con clasificación medioambiental 4.

En cuanto a la clasificación en función del tipo de combustible, los vehículos con menores emisiones de *smog* son aquellos que emplean únicamente diésel y, en mayor medida, gasolina *premium*, mientras que entre los más contaminantes destacan aquellos que emplean gasolina regular. En ninguno de los casos extremos hay vehículos que empleen etanol como combustible.